In [ ]:
import json
import boto3
import os
import requests
import string
import random
import pytz
from decimal import Decimal
from datetime import datetime as dt
from boto3.dynamodb.conditions import Attr, Key

dynamodb = boto3.resource('dynamodb', 
                          region_name = 'eu-west-2',
                          aws_access_key_id=os.environ["aws_access_key_id"],
                          aws_secret_access_key=os.environ["aws_secret_access_key"]
                         )

family = dynamodb.Table("family") #The partion key here is "_id" and sort key is "create_date"

### Helper Functions

In [ ]:
def make_id():
    """
    Create a random string that stands no chance of being repeated
    """
    bailer = str(dt.now()).replace('-', '').replace(' ', '').replace('.','').replace(':','')
    x = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16))
    val = bailer+x
    return val

def number_to_decimal(data):
    """
    Convert dictionary values from numbers to Decimal for dynamo put
    """
    try:
        return json.loads(json.dumps(data), parse_float=Decimal)
    except TypeError:
        print("E no work")
        if type(data) != dict:
            return False, "we can only work on python dictionaries."
        
        for k, v in data.items():
            if type(v) in [float, int]:
                data[k] = round(Decimal(v),2)
            elif type(v) == dict:
                number_to_decimal(v)
        return data

def db_put_item(db_name, data: dict):
    """
    - Ensure that data is bearing the applicable partition and sort keys or put will fail
    Insert new record or Update ENTIRE record. this happends by matching unique partion key data bears
    """
    status = dynamodb.Table(db_name).put_item(Item=data)
    if status['ResponseMetadata']['HTTPStatusCode'] == 200:
        return True
    return False

def now_now_str():
    return str(dt.now().astimezone(pytz.timezone("Africa/Lagos")))
def now_now():
    return dt.now().astimezone(pytz.timezone("Africa/Lagos"))

def db_get_item(db_name, d_key, d_value, many=False, baho=False):
    """
    Get an item(s) by a key: str
    Updated to keep serching till end of the table if no item is found
    many - allow for multiples records to be found, then return all found or one at random.
    baho - return all found values
    """
    table = dynamodb.Table(db_name)
    document = table.scan(FilterExpression = Attr(d_key).eq(d_value), ConsistentRead=True)
    while document["Count"] == 0:
        last = document.get("LastEvaluatedKey")
        if not last:#End of table reached
            return False, "No match found!" 
        document = table.scan(FilterExpression = Attr(d_key).eq(d_value), ConsistentRead=True, ExclusiveStartKey=last)
        
    if len(document['Items']) > 1:
        if not many:
            return False, "Multiple records found!"
        if not baho:
            data = random.choice(document["Items"])
            return True, data
        else:
            return True, document["Items"]
    return True, document['Items'][0]

In [ ]:
make_id()

In [ ]:
now_now(), now_now_str()

# Write to DynamoDB

In [ ]:
#dynamoDB is a document db.
#These docs are python dicts
a_doc = {
    "_id": make_id(),
    "surname": "Amah",
    "position": "Learner",
    "net_worth": 10000.5
}
a_doc

In [ ]:
#Insert with missing key non-decimal number
db_put_item(db_name="family", data=a_doc)

In [ ]:
#Add the missing key: create_date
a_doc["create_date"] = now_now_str()
a_doc

In [ ]:
#Insert with missing key non-decimal number/float
db_put_item(db_name="family", data=a_doc)

In [ ]:
#Convert numbers to Decimal
a_doc = number_to_decimal(a_doc)
a_doc

In [ ]:
#Insert with missing key non-decimal number/float
db_put_item(db_name="family", data=a_doc)

# Explore The Structure of a DynamoDB Document

In [ ]:
#get all documents in the table (Bad move not to filter)
document = family.scan()

In [ ]:
document

In [ ]:
document.keys()

In [ ]:
#Success query run status code is 200
document["ResponseMetadata"]["HTTPStatusCode"]

In [ ]:
document["Items"]

In [ ]:
document["ResponseMetadata"]

In [ ]:
#Total documents encountered in this scan/query, not equal to actual total items found
document["ScannedCount"]

In [ ]:
#Total found matching search query (all this time as no filters were included)
document["Count"]

In [ ]:
#This key shows up when there are more results but the single query returns data of a max of 1MB
#PS: dynamoDB document/item max size is 400kb
last = document.get("LastEvaluatedKey")

# DynamoDB Query with Filters

In [ ]:
document = family.scan(FilterExpression = Attr("surname").eq("Ojo"))

In [ ]:
#Items is a python list
document["Items"]

In [ ]:
document["ScannedCount"]

In [ ]:
document["Count"]

In [ ]:
#Update a document
ojo = document["Items"][0]
ojo

In [ ]:
ojo["company"] = "AWS"
ojo

In [ ]:
#Provide the partition and sort keys do not change, the matching doc is updated
db_put_item(db_name="family", data=ojo)

In [ ]:
#Confirm added field
document = family.scan(FilterExpression = Attr("surname").eq("Ojo"))
document["Items"][0]